<a href="https://colab.research.google.com/github/anosharahim/capstone/blob/master/emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [24]:
import pandas as pd
import os
import zipfile

In [11]:
#unzip/extract FER2013 dataset
data_path = '/content/drive/MyDrive/Capstone/datasets/archive.zip'
zip_ref = zipfile.ZipFile(data_path, 'r')
zip_ref.extractall("/tmp")
file_list =zip_ref.namelist() #save file list 
zip_ref.close()

['train/surprised/im3053.png',
 'train/surprised/im3054.png',
 'train/surprised/im3055.png',
 'train/surprised/im3056.png',
 'train/surprised/im3057.png',
 'train/surprised/im3058.png',
 'train/surprised/im3059.png',
 'train/surprised/im306.png',
 'train/surprised/im3060.png',
 'train/surprised/im3061.png',
 'train/surprised/im3062.png',
 'train/surprised/im3063.png',
 'train/surprised/im3064.png',
 'train/surprised/im3065.png',
 'train/surprised/im3066.png',
 'train/surprised/im3067.png',
 'train/surprised/im3068.png',
 'train/surprised/im3069.png',
 'train/surprised/im307.png',
 'train/surprised/im3070.png',
 'train/surprised/im3071.png',
 'train/surprised/im3072.png',
 'train/surprised/im3073.png',
 'train/surprised/im3074.png',
 'train/surprised/im3075.png',
 'train/surprised/im3076.png',
 'train/surprised/im3077.png',
 'train/surprised/im3078.png',
 'train/surprised/im3079.png',
 'train/surprised/im308.png',
 'train/surprised/im3080.png',
 'train/surprised/im3081.png',
 'train/sur